In [528]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()
import re

In [529]:
stone = pd.read_excel("../data/musicbrainz.xlsx")

In [530]:
#como hay nombres de álbumes que son números, los convertimos a string.
def num_string(dato):
    if type(dato)== int:
        return str(dato)
    else:
        return dato

In [531]:
stone["Album"] = stone["Album"].apply(num_string)

In [532]:
stone[stone["Album"] == "1999"]

,Number,Year,Album,Artist,Type,Rating,Releases,Genre
129,130,1982,1999,Prince,Album,4.25,24,rock


In [533]:
stone.Artist = stone.Artist.str.strip()
stone.Album = stone.Album.str.strip()
stone.Type = stone.Type.str.strip()
stone.Genre = stone.Genre.str.strip()

In [534]:
stone["Gen"] = stone["Genre"].str.extract(r"(^([^,])+)")[0]
stone["Subgenre"] = stone["Genre"].str.extract(r",\s*(.*$)")[0]

In [535]:
stone.sample(5)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre
204,205,1970,Tea for the Tillerman,Cat Stevens,Album,4.60,26,"folk , rock",folk,rock
389,390,1988,Surfer Rosa,Pixies,Album,4.00,9,"rock , indie",rock,indie
495,496,1998,¿Dónde están los ladrones?,Shakira,Album,4.50,4,pop,pop,NaN
252,253,1967,The Piper at the Gates of Dawn,Pink Floyd,Album,3.90,61,rock,rock,NaN
309,310,1977,Pink Flag,Wire,Album,4.25,11,punk,punk,NaN


In [536]:
stone.Album = stone.Album.str.replace(r'(Metallica...The Black Album..)',"Metallica",regex=True)
stone.Album = stone.Album.str.replace(r'(Eagles...st album.)',"Eagles",regex=True)
stone.Artist = stone.Artist.str.replace("Neil Young with Crazy Horse","Neil Young & Crazy Horse",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Bob Dylan...The Band)","Bob Dylan And The Band",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Prince\s+.*)","Prince & The Revolution",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Rufus...Chaka Khan)","Rufus",regex=True)
stone.Album = stone.Album.str.replace(r'(Proud Mary.*)',"Proud Mary",regex=True)
stone.Artist = stone.Artist.str.replace(r'(.*The Velvet Underground.*)',"The Velvet Underground",regex= True)

In [537]:
stone["Alb_url"] = stone["Album"]
stone["Art_url"] = stone["Artist"]

In [541]:
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Beatles\s..The White Album..)',"The Beatles (Remastered)",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Band...The Brown Album..)',"The Band",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^Sign.*the Times$)',"Sign 'O' The Times",regex=True)

In [542]:
stone[stone["Artist"] == "Prince"]

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
44,45,1987,Sign “☮︎” the Times,Prince,Album,4.15,14,rock,rock,NaN,Sign 'O' The Times,Prince
129,130,1982,1999,Prince,Album,4.25,24,rock,rock,NaN,1999,Prince
325,326,1980,Dirty Mind,Prince,Album,3.85,10,"funk , rock , pop",funk,"rock , pop",Dirty Mind,Prince


In [543]:
stone.sample(5)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
88,89,1997,Baduizm,Erykah Badu,Album,4.00,6,r&b,r&b,NaN,Baduizm,Erykah Badu
248,249,1985,Whitney Houston,Whitney Houston,Album,4.65,15,r&b,r&b,NaN,Whitney Houston,Whitney Houston
323,324,2002,A Rush of Blood to the Head,Coldplay,Album,4.10,24,pop,pop,NaN,A Rush of Blood to the Head,Coldplay
195,196,2010,Body Talk,Robyn,Album,NaN,7,pop,pop,NaN,Body Talk,Robyn
462,463,1968,Eli and the Thirteenth Confession,Laura Nyro,Album,NaN,10,pop,pop,NaN,Eli and the Thirteenth Confession,Laura Nyro


In [544]:
def url_prep(df, colum):
    url_replace = {"'":"%27", "&":"%26",".":"%2e","/":"%2f","#":"%23",r"(\s+)":"%20" }
    for key,value in url_replace.items():
        df[colum] = df[colum].str.replace(key,value,regex=True)

In [545]:
df = stone
colum = "Alb_url"
url_prep(df,colum)

In [546]:
colum = "Art_url"
url_prep(df,colum)

In [547]:
stone.sample(10)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
31,32,2016,Lemonade,Beyoncé,Album,4.10,12,pop,pop,NaN,Lemonade,Beyoncé
110,111,1986,Control,Janet Jackson,Album,4.50,44,"pop , r&b",pop,r&b,Control,Janet%20Jackson
323,324,2002,A Rush of Blood to the Head,Coldplay,Album,4.10,24,pop,pop,NaN,A%20Rush%20of%20Blood%20to%20the%20Head,Coldplay
292,293,1993,Last Splash,The Breeders,Album,4.00,9,"rock , alternative",rock,alternative,Last%20Splash,The%20Breeders
299,300,1997,Come On Over,Shania Twain,Album,4.00,17,country,country,NaN,Come%20On%20Over,Shania%20Twain
58,59,1972,Talking Book,Stevie Wonder,Album,5.00,19,soul,soul,NaN,Talking%20Book,Stevie%20Wonder
497,498,1977,Suicide,Suicide,Album,4.00,10,rock,rock,NaN,Suicide,Suicide
295,296,1979,Rust Never Sleeps,Neil Young & Crazy Horse,Album,4.65,12,folk,folk,NaN,Rust%20Never%20Sleeps,Neil%20Young%20%26%20Crazy%20Horse
458,459,2009,Man on the Moon: The End of Day,Kid Cudi,Album,4.40,14,hip-hop,hip-hop,NaN,Man%20on%20the%20Moon:%20The%20End%20of%20Day,Kid%20Cudi
277,278,1973,Houses of the Holy,Led Zeppelin,Album,4.50,40,rock,rock,NaN,Houses%20of%20the%20Holy,Led%20Zeppelin


In [391]:
url_replace = {"'":"%27", "&":"%26",".":"%2e","/":"%2f","#":"%23",r"(\s+)":"%20" }
url_replace[key]

for key,value in url_replace.items():
    for cel in stone["Alb_url"]:
        stone["Alb_url"]=stone["Alb_url"].str.replace(key,value,regex=True)

In [548]:
"""
stone["Alb_url"] = stone["Alb_url"].str.replace("'","%27",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("&","%26",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace(".","%2e",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("/","%2f",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("#","%23",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace(r"(\s+)","%20",regex=True)

stone["Art_url"] = stone["Art_url"].str.replace("'","%27",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("&","%26",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace(".","%2e",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("/","%2f",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("#","%23",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace(r"(\s+)","%20",regex=True)
"""

'\nstone["Alb_url"] = stone["Alb_url"].str.replace("\'","%27",regex=True)\nstone["Alb_url"] = stone["Alb_url"].str.replace("&","%26",regex=True)\nstone["Alb_url"] = stone["Alb_url"].str.replace(".","%2e",regex=True)\nstone["Alb_url"] = stone["Alb_url"].str.replace("/","%2f",regex=True)\nstone["Alb_url"] = stone["Alb_url"].str.replace("#","%23",regex=True)\nstone["Alb_url"] = stone["Alb_url"].str.replace(r"(\\s+)","%20",regex=True)\n\nstone["Art_url"] = stone["Art_url"].str.replace("\'","%27",regex=True)\nstone["Art_url"] = stone["Art_url"].str.replace("&","%26",regex=True)\nstone["Art_url"] = stone["Art_url"].str.replace(".","%2e",regex=True)\nstone["Art_url"] = stone["Art_url"].str.replace("/","%2f",regex=True)\nstone["Art_url"] = stone["Art_url"].str.replace("#","%23",regex=True)\nstone["Art_url"] = stone["Art_url"].str.replace(r"(\\s+)","%20",regex=True)\n'

In [549]:
stone.head(5)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,38,r&b,r&b,NaN,What%27s%20Going%20On,Marvin%20Gaye
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,47,rock,rock,NaN,Pet%20Sounds,The%20Beach%20Boys
2,3,1971,Blue,Joni Mitchell,Album,4.45,18,folk,folk,NaN,Blue,Joni%20Mitchell
3,4,1976,Songs in the Key of Life,Stevie Wonder,Album,4.55,24,soul,soul,NaN,Songs%20in%20the%20Key%20of%20Life,Stevie%20Wonder
4,5,1969,Abbey Road,The Beatles,Album,4.70,54,rock,rock,NaN,Abbey%20Road,The%20Beatles


In [46]:
stone.to_csv("../data/stone.csv")